In [2]:
import gym
import numpy as np
import collections
import MyNN

In [12]:
class PGAgent:
    def __init__(self, state_size, action_size, gamma=0.9,learning_rate=0.0001, load=False, load_path=''):
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate
        self.gamma = gamma
        if load:
            self.model = load_model(load_path)
        else:
            self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = MyNN.MyNN(6400)
        model.add(200, 'ReLU')
        model.add(1, 'Sigmoid')
        model.compile('Cross entropy', 'Adam')
        return model

    def act(self, state):
        state = np.reshape(state, [6400, 1])
        result = self.model.forward(state)[0][0]
        return 3 if result > np.random.rand() else 2

In [28]:
def prepro(I):
    I=I[35:195]
    I=I[::2,::2,0]
    I[I == 144] = 0
    I[I == 109] = 0
    I[I != 0] = 1
    return I.astype(np.float).reshape(6400,1)

In [29]:
def running_reward(rewards, gamma):
    result = []
    run_rew = 0
    for reward in  rewards[0,:][::-1]:
        if reward != 0:
            run_rew = 0
        run_rew = run_rew*gamma + reward
        result.append(run_rew)
    return np.hstack(result[::-1]).reshape(1,-1)

In [46]:
def play_game(agent, render=False):
    state = env.reset()
    memory = np.zeros((6400,1))
    prev_obs = np.zeros((6400,1))
    actions = np.array([])
    rewards = np.array([])
    done = False
    score = [0,0]
    while not done:
        if render:
            env.render()
        cur_obs = prepro(state)
        cur_state = cur_obs-prev_obs
        memory = np.hstack((memory, cur_state.reshape(6400,1)))
        action = agent.act(cur_state)
        actions = np.append(actions, action)
        state, reward, done, _ = env.step(action)
        rewards = np.append(rewards, reward)
        if reward == 1:
            score[1] +=1
        elif reward == -1:
            score[0] +=1
    return memory[:,1:], actions.reshape(1,-1), rewards.reshape(1,-1), score

In [62]:
def play_n_games(agent, n=20):
    X_batch = np.zeros((6400,1))
    y_batch = np.array([[0]])
    weights = np.array([[0]])
    results = [0,0]
    for i in range(n):
        a, b, c, d = play_game(agent)
        X_batch = np.hstack((X_batch, a))
        y_batch = np.hstack((y_batch, b))
        weights = np.hstack((weights, running_reward(c, agent.gamma)))
        results[0]+=d[0]
        results[1]+=d[1]
    return X_batch[:,1:], y_batch[0,1:].reshape(1,-1)-2, weights[0,1:].reshape(1,-1), np.array(results)/n

In [74]:
env = gym.make('Pong-v0')
action_size = env.action_space.n
agent = PGAgent(6400, action_size)

[2018-01-07 22:39:50,290] Making new env: Pong-v0


In [75]:
%%time
results = []
minimums = []
maximums = []
for t in range(1, 1001):
    X_batch, Y_batch, weights, result = play_n_games(agent, n=5)
    results.append(np.mean(result))
    print(t,result)
    if result[1]==21:
        break
    agent.model.optimize(X_batch, Y_batch, weights=weights,
                         lr=agent.learning_rate, num_epochs=1, report_cost=False)

1 [ 21.    0.2]
2 [ 21.   0.]
3 [ 21.    0.4]
4 [ 21.    0.8]
5 [ 21.    0.4]
6 [ 21.    0.6]
7 [ 21.    0.6]
8 [ 21.    0.6]
9 [ 21.   0.]
10 [ 21.   1.]
11 [ 21.    0.2]
12 [ 21.    0.6]
13 [ 21.    0.2]
14 [ 21.    0.4]
15 [ 21.    0.6]
16 [ 21.   1.]
17 [ 21.    0.8]
18 [ 21.    0.8]
19 [ 21.   1.]
20 [ 21.    1.2]
21 [ 21.    1.4]
22 [ 21.    1.6]
23 [ 21.    0.8]
24 [ 21.    0.8]
25 [ 21.    1.2]
26 [ 21.   1.]
27 [ 21.    0.4]
28 [ 21.    0.6]
29 [ 21.    0.4]
30 [ 21.    0.6]
31 [ 21.    0.6]
32 [ 21.    1.6]
33 [ 21.    1.6]
34 [ 21.    1.4]
35 [ 21.    0.4]
36 [ 21.    0.4]
37 [ 21.    1.2]
38 [ 21.    0.6]
39 [ 21.    0.2]
40 [ 21.    0.6]
41 [ 21.    0.4]
42 [ 21.    0.4]
43 [ 21.    1.2]
44 [ 21.   1.]
45 [ 21.    0.8]
46 [ 21.    0.6]
47 [ 21.    0.4]
48 [ 21.    0.8]
49 [ 21.    0.6]
50 [ 21.    0.4]
51 [ 21.    0.6]
52 [ 21.    0.4]
53 [ 21.    1.2]
54 [ 21.   1.]
55 [ 21.    0.4]
56 [ 21.    0.4]
57 [ 21.    0.4]
58 [ 21.    1.8]
59 [ 21.    0.4]
60 [ 21.    1.2]
61 [ 

KeyboardInterrupt: 